In [13]:
from keras.applications import VGG16, ResNet50, MobileNetV2
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Flatten, Softmax, ReLU, Dropout, Conv2D
from keras.optimizers import Adam
from keras.backend import clear_session 

from keras.preprocessing.image import ImageDataGenerator, img_to_array

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import glob
import cv2
from PIL import Image

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from Functions import process_img, get_label

import pickle

# グループプロジェクトで行なったことのまとめ(モデル作成）
# 目次

3. **学習データ集める**  
    1.画像を得る方法  
    2.ラズベリーパイの操作  
  
4. **モデル作成**  
   1.転移学習
       1.使用したモデル
       2.精度が上がらないとき
       3.モデルもメモリが大きすぎて困った時
       
   2.異常検知
       1.採用した方法
       2.閾値の設定
       3.インスタンスの保存
       4.本番ではかなりずれた
       
5. **反省**
6. **番外編**  
    1.ラズベリーパイにsklearnをインストールした  
    2.Unet  

# 学習データを集める

## 画像データを得る方法  
モデルに学習させるデータは、実際にラズベリーパイが取得するデータの方が望ましい。
なぜなら、学習させる画像の大きさや背景などが異なれば精度が落ちることが想定されるためである。
そのため、以下の手順でデータを取得した。

1. ペットボトルを置く箱を作成する
2. カメラを固定する台を作成する
3. ラズベリーパイで画像を取得する
4. 取得した画像をUSB経由でPCへ保存する

※）なお、YoLoV3などの物体検出を用いてペットボトルだけを認識させればこのようなことは必要なかったかもしれない。  
しかし、今回は時間やラズベリーパイのメモリを考慮し簡単な画像認識を採用した。

## ラズベリーパイの操作  
ラズベリーパイで写真を撮る方法は以下に示す。
1. ラズベリーパイのターミナルを起動する
2. 以下のコードを入力する  
$ raspistill -o test.png -h 400 -w 300  
実際のシステムで画像を取得する際は縦400横300で取得するため、引数で指定した。  

上記の手順で、学習に使うペットボトル画像及び誤判定に用いるダミー画像を取得した。

# モデル作成
## 転移学習
### 使用したモデル
以下のモデルを用いて転移学習をさせた。
1. VGG16（馴染みがあったので使用した）
2. ResNet50（層が深いため、より精度が上がることを期待した）
3. MobileNetV2（前期生のプロジェクトで使用されていたため）

パラメータを変えながら複数回試したが、一番良かったものを以下に示す  
結果的にはMobileNetV2が採用になった。
理由としては、精度が高くモデルの容量が小さいことが挙げられる。

In [4]:
import pandas as pd
models = ["VGG16", "ResNet50", "MobileNetV2"]
result_dict = {"weights" : ["imagenet", "imagenet", "imagenet"], "trainable" : ["all layers", "only FC", "only FC"],
               "FC layers": ["256 128 5", "256 128 5", "256 128 5"], "val_loss" :["1e-7", "2.7e-7","2.6e-5"], "val_acc" :[1.0, 1.0, 1.0]}
result_df = pd.DataFrame(result_dict, index = models)
result_df

,weights,trainable,FC layers,val_loss,val_acc
VGG16,imagenet,all layers,256 128 5,1e-7,1.0
ResNet50,imagenet,only FC,256 128 5,2.7e-7,1.0
MobileNetV2,imagenet,only FC,256 128 5,2.6e-5,1.0


In [7]:
#モデルを作成するときに使用したコードの例

clear_session()

input_tensor = Input(shape=(224, 224, 3))
mobile = MobileNetV2(include_top=False, weights="imagenet", input_tensor=input_tensor)

top_model = Sequential()
top_model.add(Flatten(input_shape=mobile.output_shape[1:]))
top_model.add(Dense(256, activation="relu"))
top_model.add(Dense(128, activation="relu"))
top_model.add(Dense(5))
top_model.add(Softmax())

mobile_model = Model(inputs=mobile.input, outputs=top_model(mobile.output))

mobile_model.compile(loss='categorical_crossentropy',
          optimizer=Adam(lr=1e-4),
          metrics=['accuracy'])


### 精度が上がらないとき

精度が上がらないときは以下を試してみた  

1. **学習率を変える**  
学習率を1e-3 ~ 1e-7まで試して精度を確認した

2. **学習する層を変える**  
転移学習させてきた重みの学習をする層を増やした。（コードは下記に記載する）

3. **FC層を増やしてみる**  
FC層を増やして学習をしてみた  

4. **モデルを変える**  
学習させるモデルを変更した  

5. **学習させるデータを増やすもしくは変更する**  
ラズベリーパイで新たに画像を取り直したりした。

以上を試してみたが、学習率を変えるのが最も簡単かつ変化が大きかった  
ただ、実際には学習させるデータを増やしたりすることを中心に行なった  
その方が、モデルの頑強性が向上すると考えたためである

In [9]:
# ファインチューニングするコード
# 以下のコードをモデルを宣言してからコンパイルする間に挿入する
mobile.trainalbe = True

### モデルのメモリが大きすぎて困ったとき

モデルは下記の「モデル保存①」のコードで重みとともに保存することができるが、そのまま保存すると200MBを超えてしまう。  
モデルの容量が大きすぎるとラズベリーパイで開くことができないという問題に直面した。  
そこで、「モデル保存②」のコードで保存し直すことで、約75MBまで落とすことができた。  
ただ、注意が必要なのはこの保存方法は再学習することができなくなり、推定専用となる。  
保存したモデルは「モデル読み込み」コードで読み込むことができる。  

In [12]:
#モデル保存①
mobile_model.save("mobile.h5")

#モデル保存②
mobile_model.save("mobile.h5", include_optimizer=False)

#モデル読み込み
model = load_model("mobile.h5")

/Users/morishuuya/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## 異常検知 
### 採用した方法
登録されていない商品は「登録されていない」と表示させるシステムを作成するにあたって、以下の方法が提案された。  
1. Softmaxの値を閾値で区切る
2. PCAのエンコーダ、デコーダ機能を使って差分をみる
3. オートエンコーダを作成し差分を見る。

1提案はSoftmaxの値が全て、０.8以下だった場合「登録されていない」と返す方法である。  
この方法はSoftmaxで出力される値が「全て自信ない」状態であれば登録されていないだろうという仮説の基成り立っている。  
しかしこれは、ダミー画像を推定した結果がとある商品で高確率を返してきたことから却下された。

2の提案は入力画像をPCAで次元圧縮を行い、そこから次元をもとに戻したものを出力する。その入力と出力の差分を測定する方法である。  
まず、学習データでPCAを学習させる。そのあと、学習データでの次元圧縮、次元復元を行い差分を見る。  
ダミーデータはPCAの学習は行わず、次元圧縮と次元復元の差分だけをみる。  
学習データとダミーデータとの間を閾値として異常検知を行う。  
ダミーデータではうまく復元ができず、差分が大きくなることを利用している。
結果的にこの方法が採用となった。

3の提案は原理としては2と同じだが、PCAを使わずオートエンコーダのモデルを作成する方法である。  
Unetを用いた方法を試行したが、ダミーデータとの差分が出ず却下となった。

### 閾値の設定
学習データの最大値とダミーデータの最小値の間を閾値として設定した。
以下はその一部を示す。

また、PCAのn_componentsは大きければ大きいほど学習データとダミーデータとの差は大きくなるが、同時にメモリも大きくなる。  
n_componentsはいろはすは40、それ以外は30で設定した。

In [23]:
# データのロード、準備
irohasu_list = glob.glob("./../dataset/version3/3_irohasu/" + "/*" + ".png")
irohasu_img = process_img(irohasu_list)
irohasu_img_re = irohasu_img.reshape(60, -1)

#ダミーデータのロード、準備
dummy_list = glob.glob("./../dataset/version3/dummy/dummy_i_" + "*" + ".png")
dummy_img_i = process_img(dummy_list)


# pcaインスタンスの作成、学習
pca_irohasu30 = PCA(n_components = 30)
pca_irohasu30.fit(irohasu_img_re)



PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [26]:
#閾値を測定するコード
#学習データの最大値、ダミーデータの最小値の間が閾値となる

#学習データ最大値の測定
s=[]
diff = 0
dataset = irohasu_img
pca_m = pca_irohasu30
for data in dataset:
    reshape = data.reshape(1, -1)
    pca_data = pca_m.transform(reshape)
    pca_redata = pca_m.inverse_transform(pca_data)
    diff = np.sum(np.abs(reshape - pca_redata))
    s.append(diff)

print("学習データの差分の最大値:", max(s).round())

#ダミーデータの最小値の測定
d_list = []
for img in dummy_img_i:
    d_res = img.reshape(1, -1)
    d_pca = pca_m.transform(d_res)
    d_pca = pca_m.inverse_transform(d_pca)
    d_diff = np.sum(np.abs(d_res - d_pca))
    d_list.append(d_diff)
print("ダミーデータの差分の最小値:", min(d_list).round())

学習データの差分の最大値: 785.0
ダミーデータの差分の最小値: 6747.0


### PCAインスタンスの保存
学習したPCAを保存し、ラズベリーパイに移行させる必要があったため、pklファイルに保存した。  
そのコードを以下に示す。  
同様に読み込みのコードも示す  

In [27]:
with open("pca_irohasu.pkl", "wb") as f:
    pickle.dump(pca_irohasu30, f) #保存
    
    
with open("pca_irohasu.pkl", "rb") as f:
            pca_irohasu30 = pickle.load(f)#読み込み

### 本番ではかなりずれた

取得した画像データをPCで処理する際にはそれほど精度は悪くなかった。  
何度か試行したが、20回に一度誤認する程度であった。  
しかし、ラズベリーパイのシステム中で取得した画像データを処理するとPCAの差分がこれまでと一桁変わってしまった。  
なので、実際の精度がかなり落ちてしまった。

# 反省点

* ラズベリーパイで取得する画像データがどのようなものなのかを確認してから、モデル作成をするべきだった。  
ラズベリーパイの取得画像の大きさ確認やカメラ角度を固定する前にモデル作成してしまったため、無駄な仕事が増えてしまった。
* ラズベリーパイが開くことのできる容量を確認すべきだった。  
モデルの容量が大きすぎてラズベリーパイが開くことができず、この解決にまた時間を要してしまった。
* pcaの閾値をラズベリーパイの取得画像で決めるべきだった  
pcaの閾値をpcで作ってしまったため、かなりずれてしまった。

# 番外編
## ラズベリーパイにsklearnをインストールした

ラズベリーパイにsklearnがインストールされてなく、pcaが使用できなかったため以下の手順でskleanをインストールした。
1. 元となる仮想環境でパッケージをtxtファイルに保存
2. 新しい仮想環境を作る
3. 新しい仮想環境でパッケージをインストール
4. tensorflow, keras, scikit-learnを個別でインストール

## unet
unetを用いたオートエンコーダにクラス分類を付随したモデルを作成してみた。  
が、精度が悪く、モデルが大きすぎるので却下した。    
供養の意味で以下にコードをのせる   
層やフィルタの数が足りない可能性が高いが、試してはいない。

In [29]:
def unet():
    clear_session()
    #インプット
    main_input = Input((224, 224, 3))
    
    #エンコード
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv1_1")(main_input)
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv1_2")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name="pool1")(conv1)
    
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv2_1")(pool1)
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv2_2")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name="pool2")(conv2)
    
    #最下層
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv3_1")(pool2)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', name="conv3_2")(conv3)
    drop3 = Dropout(0.5,name="drop3")(conv3)
    
    #デコード
    up4 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',)(UpSampling2D(size = (2,2))(drop3))
    merge4 = concatenate([conv2,up4], axis = 3)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge4)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    
    up5 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv4))
    merge5 = concatenate([conv1,up5], axis = 3)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge5)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    
    #デコードの出力
    main_out = Conv2D(3, 1, padding = 'same', kernel_initializer = 'he_normal', name = "main_out")(conv5)
    
    
    #クラス分類
    class1 = Flatten()(drop3)
    class2 = Dense(128, activation="relu")(class1)
    class_out = Dense(5, activation="softmax", name = "class_out")(class2)
    
    #モデルの作成
    model =  Model(input = main_input , output = [main_out, class_out])
    
    #モデルのコンパイル
    model.compile(optimizer = Adam(lr = 1e-4), loss = {"main_out" :"mse", "class_out" :'categorical_crossentropy'},
                  metrics = {"main_out" :"mse", "class_out" : "accuracy"})
    
    return model